# prism viability + klaeger kinome inhibition state (kis)

In [32]:
import pyreadr
def read_rds(file):
    df = pyreadr.read_r(file)[None]
    return df

viability = read_rds('../data/PRISM_klaeger_imputed_tidy.rds')
viability.head()

,depmap_id,drug,klaeger_conc,imputed_viability
0,ACH-000007,Abemaciclib,3.000000e-09,0.974677
1,ACH-000007,Abemaciclib,1.000000e-08,0.942180
2,ACH-000007,Abemaciclib,3.000000e-08,0.884299
3,ACH-000007,Abemaciclib,1.000000e-07,0.767666
4,ACH-000007,Abemaciclib,3.000000e-07,0.554302


In [33]:
kis = read_rds('../data/klaeger_full_tidy.rds')
kis['gene_name'] = 'inh_' + kis['gene_name']
kis.head()

,drug,gene_name,relative_intensity,concentration_M
0,Abemaciclib,inh_AAK1,1.000000,0.000000e+00
1,Abemaciclib,inh_AAK1,0.905556,3.000000e-09
2,Abemaciclib,inh_AAK1,0.791054,1.000000e-08
3,Abemaciclib,inh_AAK1,0.738292,3.000000e-08
4,Abemaciclib,inh_AAK1,0.639565,1.000000e-07


In [34]:
# pivot so genes are columns
kis = kis.pivot(index=['drug', 'concentration_M'], columns='gene_name', values='relative_intensity').reset_index()
kis.columns.name = None
kis.head()

,drug,concentration_M,inh_AAK1,inh_ABL1,inh_ABL2,inh_ACAD10,inh_ACAD11,inh_ACADVL,inh_ACOX1,inh_ACOX3,...,inh_ULK3,inh_UNC119,inh_VDAC3,inh_WEE1,inh_YARS,inh_YES1,inh_YTHDF3;YTHDF1,inh_YWHAG,inh_YWHAQ,inh_ZAK
0,AC-480,0.000000e+00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,AC-480,1.000000e-08,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,AC-480,3.000000e-08,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,AC-480,1.000000e-07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# create kis_viability

In [35]:
common_drugs = set(viability['drug']).intersection(set(kis['drug']))
common_conc = set(viability['klaeger_conc']).intersection(set(kis['concentration_M']))

In [36]:
kis_viability = kis[(kis['drug'].isin(common_drugs)) & (kis['concentration_M'].isin(common_conc))]
kis_viability = kis_viability.sort_values(['drug', 'concentration_M']).reset_index(drop=True)
kis_viability = kis_viability.merge(viability, left_on=['drug', 'concentration_M'], right_on=['drug', 'klaeger_conc'], how='inner')
kis_viability = kis_viability.drop(columns=['klaeger_conc'])
kis_viability = kis_viability.rename(columns={'imputed_viability': 'viability'})
kis_viability = kis_viability[['depmap_id', 'drug', 'concentration_M'] + [col for col in kis_viability.columns if col not in ['depmap_id', 'drug', 'concentration_M']]]
kis_viability.head()

,depmap_id,drug,concentration_M,inh_AAK1,inh_ABL1,inh_ABL2,inh_ACAD10,inh_ACAD11,inh_ACADVL,inh_ACOX1,...,inh_UNC119,inh_VDAC3,inh_WEE1,inh_YARS,inh_YES1,inh_YTHDF3;YTHDF1,inh_YWHAG,inh_YWHAQ,inh_ZAK,viability
0,ACH-000007,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
1,ACH-000008,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.088750
2,ACH-000011,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000
3,ACH-000012,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.298190
4,ACH-000013,AC-480,3.000000e-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.458047


In [40]:
# list of kinases to see which kinome graph to use
kinases = [col.split('inh_')[-1] for col in kis_viability.columns if col.startswith('inh_')]
kinases

['AAK1',
 'ABL1',
 'ABL2',
 'ACAD10',
 'ACAD11',
 'ACADVL',
 'ACOX1',
 'ACOX3',
 'ACP1',
 'ACSL5',
 'ACTR2',
 'ACTR3',
 'ACVR1',
 'ACVR1B',
 'ACVR2B',
 'ACVRL1',
 'ADCK1',
 'ADCK3',
 'ADCK5',
 'ADD2',
 'ADK',
 'ADRBK1',
 'ADSL',
 'AFF1',
 'AFF4',
 'AIMP1',
 'AK2',
 'AKR1C3',
 'AKT1',
 'AKT2',
 'AKT3',
 'ALK',
 'AP1B1',
 'AP1G1',
 'AP2A1',
 'AP2A2',
 'AP2B1',
 'AP2M1',
 'APRT',
 'ARAF',
 'ATR',
 'AURKA',
 'AURKB',
 'AZI2',
 'BCR',
 'BMP2K',
 'BMPR1A',
 'BMPR1B',
 'BMPR2',
 'BRAF',
 'BRD4',
 'BRD4;BRD3',
 'BTK',
 'BUB1',
 'C2CD5',
 'CAB39',
 'CABLES1',
 'CALM1',
 'CALR',
 'CAMK1G',
 'CAMK2D',
 'CAMK2G',
 'CAMK4',
 'CAMKK2',
 'CAPNS1',
 'CAPZA1',
 'CARS',
 'CBR1',
 'CCAR2',
 'CCDC47',
 'CCNA2',
 'CCNB1',
 'CCNB2',
 'CCNE1',
 'CCNE2',
 'CCNH',
 'CCNI',
 'CCNK',
 'CCNT1',
 'CCNT2',
 'CDC23',
 'CDC42BPA',
 'CDC42BPB',
 'CDC42BPG',
 'CDC7',
 'CDK1',
 'CDK10',
 'CDK12',
 'CDK13',
 'CDK16',
 'CDK17',
 'CDK18',
 'CDK2',
 'CDK3',
 'CDK4',
 'CDK5',
 'CDK6',
 'CDK7',
 'CDK9',
 'CDKL5',
 'CERS2',
 '

# kinome graph

In [38]:
import networkx
import pandas as pd

# v1: hippie
df = pd.read_csv('../data/hippieKinNetEntrez-v1.tab', sep='\t', header=None, names=['source', 'target', 'prob'])
G1 = networkx.from_pandas_edgelist(df, edge_attr=None)
assert set(df['source']).union(set(df['target'])) == set(G1.nodes)
print('nodes:', len(G1.nodes), 'edges:', len(G1.edges))

# v2: csv
df = pd.read_csv('../data/kin_unweighted.csv', sep='\t', header=None, names=['source', 'target'])
G2 = networkx.read_edgelist('../data/kin_unweighted.csv')
assert set(df['source']).union(set(df['target'])) == set(G2.nodes)
print('nodes:', len(G2.nodes), 'edges:', len(G2.edges))

# # WIP: visualize
# from pyvis.network import Network

# g = Network(cdn_resources='in_line')
# g.from_nx(G)
# g.show('ex.html')
# plot G
# networkx.draw(G, with_labels=True)

nodes: 324 edges: 1795
nodes: 517 edges: 5066


In [55]:
print('kinases:', len(kinases))
print('G1:', len(G1.nodes), 'G2:', len(G2.nodes))
print('overlap G1:', len(set(G1.nodes).intersection(set(kinases))), 'overlap G2:', len(set(G2.nodes).intersection(set(kinases))))

kinases: 520
G1: 324 G2: 517
overlap G1: 144 overlap G2: 237


In [58]:
g1 = 0
g2 = 0
for i in kinases:
    if i in G1.nodes:
        g1 += 1
    if i in G2.nodes:
        g2 += 1
print('G1:', g1, 'G2:', g2)

G1: 144 G2: 237


In [2]:
from rapidfuzz import fuzz, process

# # Example lists
# oa_drugs = df['ProductName'].str.replace(' ', '').str.replace('-', '').str.replace('/', '').str.lower()

# # Function to find matches with a similarity threshold
# def fuzzy_match_list(oa_list, master_list, threshold=80):
#     matches = []
#     for drug in oa_list:
#         # Use process.extractOne to find the best match in the master list
#         best_match = process.extractOne(drug, master_list, scorer=fuzz.ratio)
#         if best_match and best_match[1] >= threshold:
#             matches.append((drug, best_match[0], best_match[1]))  # (original, matched, score)
#     return matches

# # Find matches
# matches = fuzzy_match_list(oa_drugs, klaeger_drugs)

# # Display results
# for original, matched, score in matches:
#     print(f"{original} -> {matched} (Score: {score})")

ModuleNotFoundError: No module named 'rapidfuzz'